In [211]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
!pip install autofeat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


CTRL + F10

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
folder = ''#"drive/MyDrive/AML_Pr_1/"
raw = folder + "raw_data/"

In [3]:
raw_dataset_x = pd.read_csv(raw + 'X_train.csv',index_col=['id'])
#X = raw_dataset_x.copy()
raw_dataset_x.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,NaN,12352.094085,846.014651,105.132144,102.112809,2090.004260,2.691845,1234.374109,1000.784475,9285.751272
1.0,17757.037554,NaN,4101.016273,92959.527633,NaN,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,NaN,16198.071494,776.084467,106.385590,103.472030,2474.051881,2.287976,NaN,1012.626705,11750.284764
2.0,14226.656663,11029.642499,NaN,124055.600561,100.542483,92.860892,NaN,10492.342868,NaN,10.810076,...,10329.704431,13976.063780,737.040332,103.671234,109.458246,2656.083281,2.843706,888.353607,1048.810385,9553.922728
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,10008.251395,6212.127347,329.044233,105.084488,104.858546,1097.785204,2.732257,927.752967,1048.357330,NaN
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,10095.782015,13772.061493,NaN,NaN,100.369834,2693.053231,2.702908,1471.354073,1071.284484,9423.533063


In [4]:
raw_dataset_y = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
#y = raw_dataset_y.copy()
raw_dataset_y.head()

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0


In [5]:
just_scoring = False
if just_scoring:
  preprocessed = folder + "preprocessed_data/"
  x_train, x_test, y_train, y_test = train_test_split(raw_dataset_x, raw_dataset_y, test_size=0.2, random_state = 100)
else:
  preprocessed = folder + "ready_data/"
  x_train = pd.read_csv(raw + 'X_train.csv',index_col=['id'])
  x_test = pd.read_csv(raw + 'X_test.csv',index_col=['id'])
  y_train = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
  y_test = pd.DataFrame({'A' : []})

In [6]:
missing_values = x_train.isna().sum()
missing_values

x0       91
x1       84
x2       88
x3       91
x4       74
       ... 
x827    101
x828     95
x829     92
x830     80
x831     96
Length: 832, dtype: int64

In [7]:
for col in x_train.columns[x_train.nunique(dropna = False) <= 100]:
    print(col, " ", x_train[col].unique())

x104   [ 0. nan]
x129   [ 0. nan]
x489   [ 0. nan]
x530   [ 0. nan]


In [8]:
x_test = x_test.drop(columns = x_train.columns[x_train.nunique(dropna = False) <= 100])
x_train = x_train.drop(columns = x_train.columns[x_train.nunique(dropna = False) <= 100])

In [9]:
x_train.isnull().sum(1).sort_values(ascending=False)

id
867.0     89
430.0     87
794.0     86
565.0     85
900.0     85
          ..
627.0     43
173.0     43
1038.0    40
268.0     39
582.0     37
Length: 1212, dtype: int64

In [10]:
nans_train = x_train[x_train.columns].isna().astype(int)
nans_train = nans_train.add_prefix("isnan_")
nans_test = x_test[x_test.columns].isna().astype(int)
nans_test = nans_test.add_prefix("isnan_")

In [11]:
#X = X.fillna(X.mean())
imputer = KNNImputer(n_neighbors=3)
x_train = pd.DataFrame(imputer.fit_transform(x_train, y_train), columns = x_train.columns, index = x_train.index)
x_test = pd.DataFrame(imputer.transform(x_test), columns = x_test.columns, index = x_test.index)

In [12]:
x_train = pd.concat([x_train, nans_train], axis=1)
x_test = pd.concat([x_test, nans_test], axis=1)
x_test
x_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,isnan_x822,isnan_x823,isnan_x824,isnan_x825,isnan_x826,isnan_x827,isnan_x828,isnan_x829,isnan_x830,isnan_x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,1,0,0,0,0,0,0,0,0,0
1.0,17757.037554,10232.310907,4101.016273,92959.527633,105.293397,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,1,0,0,0,0,0,0,1,0,0
2.0,14226.656663,11029.642499,3143.476933,124055.600561,100.542483,92.860892,10435.300970,10492.342868,10346.971352,10.810076,...,0,0,0,0,0,0,0,0,0,0
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,0,0,0,0,0,0,0,0,0,1
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,18707.457475,13610.725702,3785.886941,113497.632841,109.526764,97.812339,11274.011935,10803.953566,10949.811419,11.024856,...,0,0,0,0,0,0,0,0,1,0
1208.0,17108.239122,12168.536128,3442.619145,98218.773311,100.468476,109.994258,11031.326117,10231.743317,10687.321177,10.238957,...,0,0,0,0,0,0,0,0,0,0
1209.0,14264.707321,9273.405761,3580.894003,101668.927699,102.620705,104.470375,9159.594864,10661.827392,10623.176915,10.132906,...,0,0,0,0,0,0,1,0,0,0


In [13]:
x_train.describe().transpose().sort_values('min')

,count,mean,std,min,25%,50%,75%,max
x173,1212.0,-1.852303e+22,2.496973e+22,-2.937933e+23,-2.871468e+22,-1.414654e+22,-3.972937e+21,7.002915e+22
x665,1212.0,4.526528e+22,3.966675e+22,-6.392632e+22,2.704526e+22,3.921405e+22,5.853573e+22,9.935455e+23
x596,1212.0,-1.835249e+14,1.234405e+14,-8.756492e+14,-2.335186e+14,-1.486501e+14,-1.082350e+14,2.926746e+14
x132,1212.0,-1.386441e+06,2.086794e+05,-2.819624e+06,-1.498551e+06,-1.367610e+06,-1.278865e+06,-5.738037e+05
x374,1212.0,-5.416921e+05,7.255622e+04,-1.002561e+06,-5.818485e+05,-5.352921e+05,-5.053880e+05,-1.607672e+05
...,...,...,...,...,...,...,...,...
x467,1212.0,1.052099e+06,2.765164e+04,1.000249e+06,1.029032e+06,1.053533e+06,1.073040e+06,1.099984e+06
x421,1212.0,1.049927e+06,2.740026e+04,1.000304e+06,1.027643e+06,1.049873e+06,1.072518e+06,1.099971e+06
x637,1212.0,1.049070e+06,2.763131e+04,1.000329e+06,1.026365e+06,1.047396e+06,1.072268e+06,1.099951e+06
x331,1212.0,1.049137e+06,2.841760e+04,1.000372e+06,1.024670e+06,1.049827e+06,1.070840e+06,1.099887e+06


In [14]:
s = StandardScaler()
x_train[[str(col) for col in x_train.columns]] = s.fit_transform(x_train)
x_test[[str(col) for col in x_train.columns]] = s.transform(x_test)

In [15]:
x_train.to_csv(preprocessed + 'X_train_preprocessed_DL.csv',index_label="id")
x_test.to_csv(preprocessed + 'X_test_preprocessed_DL.csv',index_label="id")
y_train.to_csv(preprocessed + 'y_train_preprocessed_DL.csv',index_label="id")
y_test.to_csv(preprocessed + 'y_test_preprocessed_DL.csv',index_label="id")

In [16]:
!pip install yellowbrick

In [18]:
%%time
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_selection import RFECV, SequentialFeatureSelector, SelectFromModel
from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared, RBF

from sklearn.model_selection import StratifiedKFold

# from yellowbrick.model_selection import RFECV
# from yellowbrick.datasets import load_credit

rfr = ExtraTreesRegressor(n_jobs=-1, n_estimators=600, random_state=0)
#rfr = GaussianProcessRegressor(kernel=RBF(), random_state=0)
#selector = RFECV(estimator = rfr, n_jobs=-1)
selector = SequentialFeatureSelector(estimator = rfr, n_features_to_select=3, tol=0, n_jobs=-1)
selector.fit(X = np.array(x_train), y = np.ravel(y_train))
#CPU times: total: 9.83 s
#Wall time: 16min 54s

CPU times: total: 9.83 s
Wall time: 16min 54s


SequentialFeatureSelector(estimator=ExtraTreesRegressor(n_estimators=600,
                                                        n_jobs=-1,
                                                        random_state=0),
                          n_features_to_select=1, n_jobs=-1, tol=0)

In [34]:
selector.get_support().shape

(1656,)

In [33]:
np.argwhere(selector.get_support())
#array([[1534]], dtype=int64)

array([[1534]], dtype=int64)

In [27]:
x_train.iloc(selector.get_support())

TypeError: unhashable type: 'numpy.ndarray'

In [32]:
selector.transform(x_train)
#-0.29824283 (every element)

C:\Users\david\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SequentialFeatureSelector was fitted without feature names
  warnings.warn(


array([[-0.29824283],
       [-0.29824283],
       [-0.29824283],
       ...,
       [-0.29824283],
       [-0.29824283],
       [-0.29824283]])

In [38]:
x_train.iloc[:, 1534]
#isnan_x710

id
0.0      -0.298243
1.0      -0.298243
2.0      -0.298243
3.0      -0.298243
4.0      -0.298243
            ...   
1207.0   -0.298243
1208.0   -0.298243
1209.0   -0.298243
1210.0   -0.298243
1211.0   -0.298243
Name: isnan_x710, Length: 1212, dtype: float64

In [ ]:
(x_train.iloc[:, 1534]==3.35297245).sum()

In [43]:
x_train.iloc[:, 1534].unique()
#array([-0.29824283,  3.35297245])

array([-0.29824283,  3.35297245])

In [22]:
np.absolute(rfecv.estimator_.coef_)

NameError: name 'rfecv' is not defined

In [ ]:
# %%time
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.feature_selection import RFE
# from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared, RBF
# rfr = ExtraTreesRegressor(n_jobs=-1, max_depth=None, n_estimators=341, random_state=0, min_samples_split=3, max_features=None)
# rfr = GaussianProcessRegressor(kernel=RBF(), random_state=0)
# selector = RFE(estimator = f_regression, n_features_to_select = 100)
# selector.fit(X = np.array(x_train), y = np.ravel(y_train))

In [ ]:

#selector.show()

In [231]:
model=IsolationForest(n_estimators=150,max_samples='auto',contamination=float(0.2),random_state=np.random.RandomState(42), n_jobs=-1)
model.fit(np.array(x_train))
is_anomaly = model.predict(np.array(x_train))
x_train['anomaly'] = is_anomaly
y_train['anomaly'] = is_anomaly

In [232]:
x_train['anomaly'].value_counts()

 1    969
-1    243
Name: anomaly, dtype: int64

In [233]:
y_train['anomaly'].value_counts()

 1    969
-1    243
Name: anomaly, dtype: int64

In [234]:
x_train = x_train[x_train.anomaly > 0]
y_train = y_train[y_train.anomaly > 0]

In [235]:
x_train = x_train.drop('anomaly',axis=1)
y_train = y_train.drop('anomaly',axis=1)

In [236]:
x_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,isnan_x822,isnan_x823,isnan_x824,isnan_x825,isnan_x826,isnan_x827,isnan_x828,isnan_x829,isnan_x830,isnan_x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,-0.439786,-0.259312,-0.226678,-0.583575,-0.971490,-0.788129,1.130952,1.293719,1.249786,0.165677,...,3.530817,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
1.0,1.160629,-0.444346,1.587980,-0.718887,0.084219,-0.013704,-0.013749,1.147812,-1.427827,1.379006,...,3.530817,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,3.489114,-0.265841,-0.293294
2.0,-0.413991,0.078692,-0.625908,2.591201,-1.641252,-0.768303,0.426802,-0.074594,-0.475411,0.749991,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
3.0,-2.849547,-2.312666,-2.929110,0.047340,-0.074979,0.210041,0.023987,-0.048343,0.150672,0.029593,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,3.409545
4.0,-0.603835,1.548004,-0.013582,-0.815860,-0.472779,-0.452370,-0.376156,0.870522,0.480380,1.950737,...,-0.283221,-0.272888,3.428979,3.409545,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,1.584535,1.771847,0.859382,1.467335,1.621723,-0.234101,1.303750,1.064966,1.644260,0.965497,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,3.761649,-0.293294
1208.0,0.871252,0.825791,0.065727,-0.159055,-1.668130,1.080184,1.050000,-1.027606,0.721307,0.176938,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
1209.0,-0.397020,-1.073375,0.385426,0.208204,-0.886469,0.484223,-0.907064,0.545211,0.495767,0.070529,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,3.428979,-0.286606,-0.265841,-0.293294


In [237]:
y_train.size

969

In [238]:
# %%time
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF
# trained_model = GaussianProcessRegressor(kernel = RBF())
# for _ in range(y_train.size):
#   trained_model.fit(x_train, y_train)
#   importances = trained_model.feature_importances_
#   std = np.std([trained_model.feature_importances_ for trained_model in trained_model.estimators_], axis=0)
#   indices = np.argsort(importances)[::-1]

In [ ]:
from sklearn.feature_selection import RFE


In [239]:
# selector = VarianceThreshold()
# x_train_tmp = selector.fit_transform(X,y)

selector = SelectKBest(f_regression, k=80)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)
x_train = x_train.iloc[:,cols]
x_test = x_test.iloc[:,cols]

#dataset_selected_x


In [240]:
#new_X.describe()

In [241]:
#x_train_tmp = pd.DataFrame()

In [242]:
# fsel = FeatureSelector(
#         problem_type='regression',
#         featsel_runs=10,
#         keep=None,
#         n_jobs=-1,
#         verbose=1,)

# new_X = fsel.fit_transform(x_train_tmp, y)
# autofeat_cols = new_X.columns.values
# autofeat_cols

In [243]:
# for j in range(832):
#         if j in accepted:
#           print (j)
#         else:
#           del dataset_selected_x[str(j)]
#dataset_selected_x = dataset_selected_x.iloc[:,cols]
# train_stats = dataset_selected_x.describe()
# train_stats = train_stats.transpose()
#dataset_selected_x

In [ ]:
x_train.to_csv(preprocessed + 'X_train_preprocessed.csv',index_label="id")
x_test.to_csv(preprocessed + 'X_test_preprocessed.csv',index_label="id")
y_train.to_csv(preprocessed + 'y_train_preprocessed.csv',index_label="id")
y_test.to_csv(preprocessed + 'y_test_preprocessed.csv',index_label="id")

In [245]:
x_train.columns

Index(['x23', 'x27', 'x107', 'x113', 'x115', 'x133', 'x146', 'x159', 'x172',
       'x194', 'x200', 'x203', 'x209', 'x213', 'x214', 'x231', 'x233', 'x242',
       'x254', 'x276', 'x286', 'x287', 'x306', 'x309', 'x310', 'x315', 'x319',
       'x320', 'x325', 'x327', 'x334', 'x349', 'x362', 'x399', 'x410', 'x415',
       'x437', 'x440', 'x445', 'x452', 'x458', 'x465', 'x484', 'x485', 'x496',
       'x507', 'x517', 'x531', 'x538', 'x542', 'x543', 'x546', 'x558', 'x571',
       'x594', 'x612', 'x613', 'x633', 'x640', 'x641', 'x654', 'x657', 'x671',
       'x672', 'x675', 'x681', 'x690', 'x702', 'x711', 'x712', 'x720', 'x721',
       'x734', 'x745', 'x748', 'x766', 'x769', 'x777', 'x801', 'x817'],
      dtype='object')